## Load Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, plot_roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from tqdm.notebook import tqdm

## Load Data

In [ ]:
pre_norm_first_gene_counts_df = pd.read_csv("original_and_validation_below_ct_30_vst.csv",low_memory=False).T
first_headers = pre_norm_first_gene_counts_df.iloc[0]
pre_norm_first_gene_counts_df = pre_norm_first_gene_counts_df[1:]
pre_norm_first_gene_counts_df.columns = first_headers

In [ ]:
sample_IDs = pd.read_csv("original_and_validation_below_ct_30_vst.csv",low_memory=False).columns[1:]

In [ ]:
ENSG_trans = pd.read_csv('gene2name.txt',low_memory=False, index_col = 0, header = None, sep = '\t').T





In [ ]:
with open('gene2name.txt', 'r') as f:
    E2N = {}
    for line in f:
        ecode, name, _ = line.split()
        E2N[ecode] = name

N2E = {v: k for k,v in E2N.items()}

In [ ]:
I2E = dict(first_headers)
E2I = {v: k for k,v in I2E.items()}

In [ ]:
I2N = {v: E2N[k] for v, k in I2E.items()}
N2I = {v: k for k,v in I2N.items()}

In [ ]:
scaler = StandardScaler(with_std = True)
scaler.fit(pre_norm_first_gene_counts_df)
first_gene_counts_df = scaler.transform(pre_norm_first_gene_counts_df)

In [ ]:
UCSF_meta = pd.read_csv('UCSF_samples_metadata.csv', index_col = 'czb_id').loc[sample_IDs]

In [ ]:
first_second_SC2_3way = UCSF_meta['viral_status']
first_second_SC2_3way = np.char.capitalize(first_second_SC2_3way.values.astype('str'))
first_second_SC2_3way[first_second_SC2_3way == 'Other_virus'] = 'Other virus'
first_second_SC2_3way[first_second_SC2_3way == 'No_virus'] = 'No virus'

first_second_SC2 = first_second_SC2_3way == 'Sc2'

rpm = UCSF_meta['sc2_rpm'].values

In [ ]:
rpm_first_gene_counts_df = np.concatenate((first_gene_counts_df, rpm.reshape((-1, 1))), axis = 1)

## Split Data into Training and Testing Cohorts

In [ ]:
seed = 8675309

In [ ]:
#Splits the data into training and validation datasets (70:30)

[train_ind, test_ind] = train_test_split(np.arange(318), random_state = seed, test_size = 0.3
                                         , stratify = 2*first_second_SC2
                                        )

TRcomb_counts = first_gene_counts_df[train_ind]
TEcomb_counts = first_gene_counts_df[test_ind]





TRcomb_SC2 = first_second_SC2[train_ind]
TEcomb_SC2 = first_second_SC2[test_ind]


rpm_TRcomb_counts = rpm_first_gene_counts_df[train_ind]
rpm_TEcomb_counts = rpm_first_gene_counts_df[test_ind]



#Uncommment to use data without centering and scaling


#TRcomb_counts = pre_norm_first_gene_counts_df.values[train_ind]
#TEcomb_counts = pre_norm_first_gene_counts_df.values[test_ind]

#rpm_TRcomb_counts = rpm_first_gene_counts_df[train_ind]
#rpm_TEcomb_counts = rpm_first_gene_counts_df[test_ind]


## Generate 2-gene sets

In [ ]:
#Given the current gene set, finds the top 3 genes by how much they increase performance when added to the model.

def next_three(genes):
    
    
    first_all_scores = []

    for i in range(TRcomb_counts.shape[1]):
        if not i%100: print(i)
        clf = SVC(gamma = 'auto', probability = True)
        score = cross_validate(clf, TRcomb_counts[:, genes + [i]], TRcomb_SC2, scoring='roc_auc', cv=StratifiedKFold(n_splits=5, shuffle=True))
        av = np.average(score['test_score'])
        first_all_scores.append(av)
    
    
    x = -2000
    barset = np.argsort(first_all_scores)[x:]
    bar = first_all_scores[barset[0]]
    
    
    second_all_scores = []

    for i in barset:
        cur_scores = []
        print(i)
        for j in range(10):
            clf = SVC(gamma = 'auto', probability = True)
            score = cross_validate(clf, TRcomb_counts[:, genes + [i]], TRcomb_SC2, scoring='roc_auc', cv=StratifiedKFold(n_splits=5, shuffle=True))
            av = np.average(score['test_score'])
            if av < bar:
                cur_scores = np.zeros(10)
                break
            cur_scores.append(av)
            
        second_all_scores.append(cur_scores)
    
    
    onlyset = np.where(np.average(second_all_scores, axis = 1) != 0)[0]
    
    
    third_all_scores = []
    for i in barset[onlyset]:
        cur_scores = []
        print(i)
        for j in range(100):
            clf = SVC(gamma = 'auto', probability = True)
            score = cross_validate(clf, TRcomb_counts[:, genes + [i]], TRcomb_SC2, scoring='roc_auc', cv=StratifiedKFold(n_splits=5, shuffle=True))
            av = np.average(score['test_score'])
            cur_scores.append(av)
        third_all_scores.append(cur_scores)
    
    return barset[onlyset][np.argpartition(np.average(third_all_scores, axis = 1), -3)[-3:]]

In [ ]:
next_three([])

In [ ]:
next_three([6589])

In [ ]:
next_three([6487])

In [ ]:
next_three([5043])

## Evaluate performance of resulting gene sets

In [ ]:
#Given a gene set, generates the following five scores:
#
#   CV score on the training cohort
#   CV score on the testing cohort
#   Score on the testing cohort when trained on the training cohort
#   CV score on the training cohort with rpm
#   Score on the testing cohort when trained on the training cohort with rpm

def five_scores(gene_set):
    
    print(ENSG_trans[first_headers[gene_set]].iloc[0].values)
    
    
    cur_scores = []
    for i in range(10000):
        clf = SVC(gamma = 'auto', probability = True)
        score = cross_validate(clf, TRcomb_counts[:, gene_set], TRcomb_SC2, scoring='roc_auc', cv=StratifiedKFold(n_splits=5, shuffle=True))
        av = np.average(score['test_score'])
        cur_scores.append(av)
    
    print(np.average(cur_scores), np.std(cur_scores))
    
    
    
    cur_scores = []
    for i in range(10000):
        clf = SVC(gamma = 'auto', probability = True)
        score = cross_validate(clf, TEcomb_counts[:, gene_set], TEcomb_SC2, scoring='roc_auc', cv=StratifiedKFold(n_splits=5, shuffle=True))
        av = np.average(score['test_score'])
        cur_scores.append(av)
    
    print(np.average(cur_scores), np.std(cur_scores))
    
    
    
    X = TRcomb_counts[:, gene_set]
    y = TRcomb_SC2
    clf = SVC(gamma = 'auto', probability = True)
    clf.fit(X, y)
    print(roc_auc_score(TEcomb_SC2, clf.predict_proba(TEcomb_counts[:, gene_set])[:, 1]))
    
    
    
    cur_scores = []
    for i in range(10000):
        clf = SVC(gamma = 'auto', probability = True)
        score = cross_validate(clf, rpm_TRcomb_counts[:, gene_set + [-1]], TRcomb_SC2, scoring='roc_auc', cv=StratifiedKFold(n_splits=5, shuffle=True))
        av = np.average(score['test_score'])
        cur_scores.append(av)
    
    print(np.average(cur_scores), np.std(cur_scores))
    
    
    
    X = rpm_TRcomb_counts[:, gene_set + [-1]]
    y = TRcomb_SC2
    clf = SVC(gamma = 'auto', probability = True)
    clf.fit(X, y)
    print(roc_auc_score(TEcomb_SC2, clf.predict_proba(rpm_TEcomb_counts[:, gene_set + [-1]])[:, 1]))

In [ ]:




for gene_set in [[6589, 12501], [6589, 619], [6589, 8318],
                [6487, 7921], [6487, 10579], [6487, 8318],
                [5043, 8318], [5043, 10097], [5043, 11895]]:
    
    five_scores(gene_set)

In [ ]:
all_text = ['IFI6, GRINA', 'IFI44L, PTAFR', 'IFI6, C15orf48', 'IFI6, GBP5', 'IFI44L, GBP5']
all_inds = [[5043, 11895], [6487, 10579], [5043, 10097], [5043, 8318], [6487, 8318]]
fig, ax = plt.subplots(figsize=(6, 6))
for i in range(5):
    text = all_text[i]
    comb_inds = all_inds[i]
    clf = SVC(gamma = 'auto', probability = True)
    clf.fit(rpm_TRcomb_counts[:, comb_inds], TRcomb_SC2)
    curve = plot_roc_curve(clf, rpm_TEcomb_counts[:, comb_inds], TEcomb_SC2, ax = ax)
    #curve.line_.set_color('darkorange')
    curve.line_.set_label(text+' (AUC = %.2f)' % curve.roc_auc)

ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='navy', alpha=0.8)
ax.legend(loc = 4, bbox_to_anchor=(1.62, 0))
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
plt.title('ROC curve')
plt.savefig('Figure 1b.pdf', bbox_inches = 'tight', dpi = 200)
plt.show()

## Generate Pairplots

In [ ]:
data = pd.DataFrame(first_gene_counts_df, columns = first_headers)
data['Status'] = first_second_SC2_3way
data['SC'] = np.array(['Sc2 Neg', 'Sc2 Pos'])[(first_second_SC2_3way == 'Sc2')+0]


In [ ]:
genes = ['IFI6', 'GBP5']

sns.set(font_scale=1)

pal = dict(zip(data.Status.unique(),['#1180ff', 'gray', 'red']))

g = sns.pairplot(data, vars = [N2E[n] for n in genes], hue = 'Status', palette = pal, plot_kws = dict(alpha = 0.5), diag_kws=dict(common_norm = False), aspect = 1.1)

replacements = dict(zip([N2E[n] for n in genes], genes))

for i in range(len(genes)):
    for j in range(len(genes)):
        xlabel = g.axes[i][j].get_xlabel()
        ylabel = g.axes[i][j].get_ylabel()
        if xlabel in replacements.keys():
            g.axes[i][j].set_xlabel(replacements[xlabel], fontsize = 30)
        if ylabel in replacements.keys():
            g.axes[i][j].set_ylabel(replacements[ylabel], rotation = 0, horizontalalignment = 'right', verticalalignment = 'center', fontsize = 30)

plt.setp(g._legend.get_texts(), fontsize='20', verticalalignment='center')
plt.setp(g._legend.get_title(), fontsize='30')
plt.setp(g._legend, bbox_to_anchor=(1.3, 0.5))

for line in g._legend.legendHandles:
    line.set_sizes([300, 300])

tbbox = g.fig.get_tightbbox(g.fig.canvas.get_renderer())
g.savefig('Figure 2a.pdf', bbox_inches = tbbox, dpi = 200)

In [ ]:
genes = ['IFI6', 'GBP5']

sns.set(font_scale=1)

pal = dict(zip(data.SC.unique(),['#1180ff', 'gray']))

g = sns.pairplot(data, vars = [N2E[n] for n in genes], hue = 'SC', palette = pal, plot_kws = dict(alpha = 0.5), diag_kws=dict(common_norm = False), aspect = 1.1)

replacements = dict(zip([N2E[n] for n in genes], genes))

for i in range(len(genes)):
    for j in range(len(genes)):
        xlabel = g.axes[i][j].get_xlabel()
        ylabel = g.axes[i][j].get_ylabel()
        if xlabel in replacements.keys():
            g.axes[i][j].set_xlabel(replacements[xlabel], fontsize = 30)
        if ylabel in replacements.keys():
            g.axes[i][j].set_ylabel(replacements[ylabel], rotation = 0, horizontalalignment = 'right', verticalalignment = 'center', fontsize = 30)

plt.setp(g._legend.get_texts(), fontsize='20', verticalalignment='center')
plt.setp(g._legend.get_title(), text="Status", fontsize='30')
plt.setp(g._legend, bbox_to_anchor=(1.3, 0.5))

for line in g._legend.legendHandles:
    line.set_sizes([300, 300])

g.savefig('Figure 1b.pdf', bbox_inches = tbbox, dpi = 200)

## Test on Cornell Data

In [ ]:
cornell_counts = pd.read_csv('Cornell_counts/cornell_subset_vst.csv')
cornell_counts.index = cornell_counts['Unnamed: 0']
cornell_counts = cornell_counts.drop('Unnamed: 0', axis = 1)

In [ ]:
cornell_metatable = pd.read_csv('Cornell_counts/NewYork_samples_metadata.csv')

In [ ]:
corn_SC2 = []

for sid in cornell_counts.columns:
    corn_SC2.append(cornell_metatable.loc[cornell_metatable['SampleID'] == sid]['Class'].iloc[0] == 'Positive')

In [ ]:
corn_scaler = StandardScaler()
corn_scaler.fit(cornell_counts.T)
scaled_cornell_counts = corn_scaler.transform(cornell_counts.T)

In [ ]:
#Loops over a number of gene sets to generate the following two scores:
#
#   CV score on the validation cohort
#   Score on the validation cohort when trained on the training cohort

all_pairs = [['HERC6', 'COA3'], 
             ['HERC6', 'TNIP3'], 
             ['HERC6', 'GBP5'], 
             ['IFI44L', 'FCGR1A'], 
             ['IFI44L', 'PTAFR'], 
             ['IFI44L', 'GBP5'], 
             ['IFI6', 'GBP5'], 
             ['IFI6', 'C15orf48'], 
             ['IFI6', 'GRINA']]




display = []

for gene_set in tqdm(all_pairs, leave=False):
    
    gene_set_n = list(map(N2I.get, gene_set))
    names = '{}, {}'.format(*gene_set)
    
    cur_scores = []
    cornell_set = []

    
    for gene in gene_set:
        cornell_set.append(cornell_counts.index.get_loc(gene))
    
    for i in tqdm(range(10000), leave=False):
        clf = SVC(gamma = 'auto', probability = True)
        score = cross_validate(clf, scaled_cornell_counts[:, cornell_set], corn_SC2, scoring='roc_auc', cv=StratifiedKFold(n_splits=5, shuffle=True))
        av = np.average(score['test_score'])
        cur_scores.append(av)
    
    score1 = "{:.3f} ({:.3f})".format(np.average(cur_scores), np.std(cur_scores))
    
    
    clf = SVC(gamma = 'auto')
    clf.fit(TRcomb_counts[:, gene_set_n], TRcomb_SC2)
    score2 = "{:.3f}".format(roc_auc_score(corn_SC2, clf.decision_function(scaled_cornell_counts[:,cornell_set])))
    
    display.append([names, score1, score2])
    
    
pd.DataFrame(display, columns = ['Genes', '5-fold CV', 'Trained on 70%'])

In [ ]:
#Loops over a number of gene sets to generate the following two scores:
#
#   CV score on the validation cohort
#   Score on the validation cohort when trained on the training cohort

all_pairs = [['HERC6', 'COA3'], 
             ['HERC6', 'TNIP3'], 
             ['HERC6', 'GBP5'], 
             ['IFI44L', 'FCGR1A'], 
             ['IFI44L', 'PTAFR'], 
             ['IFI44L', 'GBP5'], 
             ['IFI6', 'GBP5'], 
             ['IFI6', 'C15orf48'], 
             ['IFI6', 'GRINA']]

display = []

for gene_set in tqdm(all_pairs, leave=False):
    
    gene_set_n = list(map(N2I.get, gene_set))
    names = '{}, {}'.format(*gene_set)
    
    cur_scores = []
    cornell_set = []

    
    for gene in gene_set:
        cornell_set.append(cornell_counts.index.get_loc(gene))
    
    #for i in tqdm(range(10000), leave=False):
    #    clf = SVC(gamma = 'auto', probability = True)
    #    score = cross_validate(clf, scaled_cornell_counts[:, cornell_set], corn_SC2, scoring='roc_auc', cv=StratifiedKFold(n_splits=5, shuffle=True))
    #    av = np.average(score['test_score'])
    #    cur_scores.append(av)
    #
    #score1 = "{:.3f} ({:.3f})".format(np.average(cur_scores), np.std(cur_scores))
    
    
    clf = SVC(gamma = 'auto', probability = True)
    clf.fit(TRcomb_counts[:, gene_set_n], TRcomb_SC2)
    score2 = "{:.3f}".format(roc_auc_score(corn_SC2, clf.decision_function(scaled_cornell_counts[:,cornell_set])))
    
    #display.append([names, score1, score2])
    display.append([names, score2])
    
    
#pd.DataFrame(display, columns = ['Genes', '5-fold CV', 'Trained on 70%'])
pd.DataFrame(display, columns = ['Genes', 'Trained on 70%'])

In [ ]:
#Generate single gene and 2-gene AUC scores for comparison with fold-change

def all_gene_scores(add_set):
    
    all_scores = []
    
    
    cornell_set = []
    
    for gene in add_set:
        cornell_set.append(cornell_counts.index.get_loc(gene))
    
    
    for i in range(scaled_cornell_counts.shape[1]):
        cur_scores = []
        if not i%100: print(i)
        for j in range(5):
            clf = SVC(gamma = 'auto', probability = True)
            score = cross_validate(clf, scaled_cornell_counts[:, [i]+cornell_set], corn_SC2, scoring='roc_auc', cv=StratifiedKFold(n_splits=5, shuffle=True))
            av = np.average(score['test_score'])
            cur_scores.append(av)
            
        all_scores.append(cur_scores)
    
    avs = pd.DataFrame(cornell_counts.index.values, columns = ['Gene'])
    avs['Score'] = np.average(all_scores, axis = 1)
    
    if add_set == []:
        np.savetxt("new_york_one_gene_scores.csv", avs.sort_values('Score', ascending = False), delimiter=",", fmt='%s')
    else:
        np.savetxt("new_york_scores_with_"+"_".join(add_set)+".csv", avs.sort_values('Score', ascending = False), delimiter=",", fmt='%s')

In [ ]:
all_gene_scores([])

In [ ]:
all_gene_scores(['IFI6'])

In [ ]:
all_gene_scores(['IFI44L'])

In [ ]:
all_gene_scores(['HERC6'])